In [1]:
import pandas as pd 
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as tranforms

In [3]:
training_data = datasets.FashionMNIST(root='mnist',train=True,download=False,transform=tranforms.ToTensor())
testing_data = datasets.FashionMNIST(root='mnist',train=False,download=False,transform=tranforms.ToTensor())

In [5]:
batch_size = 64

In [6]:
train_loader = DataLoader(training_data,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(testing_data,batch_size=batch_size,shuffle=True)

In [10]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [11]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.stack = nn.Sequential(nn.Linear(28*28,512),
                                   nn.ReLU(),
                                   nn.Linear(512,512),
                                   nn.ReLU(),
                                   nn.Linear(512,10))
    def forward(self,x):
        x = self.flatten(x)
        logits = self.stack(x)
        return logits

In [12]:
model = NN().to(device)

In [13]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [14]:
def train_model(dataloader,model,loss,optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch ,(X,y) in enumerate(dataloader):
        X,y = X.to(device),y.to(device)
        pred = model(X)
        model_loss = loss(pred,y)
        model_loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss1, current = model_loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss1:>7f}  [{current:>5d}/{size:>5d}]")

In [15]:
def test_model(dataloader,model,loss):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss,correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device),y.to(device)
            pred = model(X)
            test_loss += loss(pred,y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    return test_loss,correct

In [16]:
epochs = 10
for i in range(epochs):
    print(f"Epoch {i+1}\n-------------------------------")
    train_model(train_loader, model, loss, optimizer)
    eval_loss,accuracy = test_model(test_loader, model, loss)
    print(f"The evaluation loss {eval_loss}")
    print(f"The accuracy is {round(accuracy,2)}\n")

Epoch 1
-------------------------------
loss: 2.304830  [   64/60000]
loss: 0.808105  [ 6464/60000]
loss: 0.518686  [12864/60000]
loss: 0.591237  [19264/60000]
loss: 0.408907  [25664/60000]
loss: 0.314740  [32064/60000]
loss: 0.354824  [38464/60000]
loss: 0.362383  [44864/60000]
loss: 0.491875  [51264/60000]
loss: 0.332160  [57664/60000]
The evaluation loss 0.42653718116177114
The accuracy is 0.84

Epoch 2
-------------------------------
loss: 0.236421  [   64/60000]
loss: 0.445067  [ 6464/60000]
loss: 0.341554  [12864/60000]
loss: 0.433017  [19264/60000]
loss: 0.449673  [25664/60000]
loss: 0.418769  [32064/60000]
loss: 0.260712  [38464/60000]
loss: 0.608624  [44864/60000]
loss: 0.510515  [51264/60000]
loss: 0.331816  [57664/60000]
The evaluation loss 0.3740062281774108
The accuracy is 0.87

Epoch 3
-------------------------------
loss: 0.262375  [   64/60000]
loss: 0.449440  [ 6464/60000]
loss: 0.453927  [12864/60000]
loss: 0.411219  [19264/60000]
loss: 0.293225  [25664/60000]
loss: 0

In [17]:
torch.save(model.state_dict(),'model1.pth')

In [18]:
classes = training_data.classes

In [19]:
model.eval()

NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [29]:
x,y = testing_data[0][0],testing_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    output = classes[pred.argmax()]
    print(f"The predicted class is {output} and the actual class is {classes[y]}.")

The predicted class is Ankle boot and the actual class is Ankle boot.
